In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datalab_utils
import astropy.units as u

from dl import authClient as ac, queryClient as qc
from collections import Counter
from tqdm import tqdm

plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
fig_dir = "/dlusers/adriansh/work/analyses/plots/strings/"

In [2]:
lc_df = pd.concat([pd.read_parquet(f"/dlusers/adriansh/work/analyses/results/strings/june12_2024/lightcurves_batch{i}.parquet") for i in range(10)])
time_df = pd.concat([pd.read_parquet(f"/dlusers/adriansh/work/analyses/results/strings/june12_2024/times_batch{i}.parquet") for i in range(10)])

In [ ]:
bandwidths = np.arange(1, 13) / 100
params = dict(achromatic = True, 
              factor_of_two = True)
bg_df = lc_df.drop(columns="cluster_label", axis=0)
bg_df = bg_df.reset_index(drop=True)
n_bg = []

for bw in tqdm(bandwidths):
    background_df = datalab_utils.cluster_label_dataframe(bg_df, bandwidth=bw)
    background_df = background_df.groupby(by="objectid").filter(datalab_utils.unstable_filter)
    g = background_df.groupby("objectid")
    background_df = g.filter(lambda group: datalab_utils.lens_filter(group, **params))
    n_bg.append(background_df["objectid"].unique().size)